In [34]:
import mne
import os
import autoreject as aj
import numpy as np
import pandas as pd

In [35]:
# Declaring the general path
path = os.path.join('C:/Users/annad/Documents/Thesis/Data/') 
subj_names = []

# Extracting subject name based on the file name
for x in os.listdir(path+'Post ICA/'):
    if 'raw.fif' in x:
        subj_names.append(x.split('_')[2])

print(subj_names)

['VP10', 'VP11', 'VP12', 'VP13', 'VP14', 'VP15', 'VP16', 'VP17', 'VP18', 'VP19', 'VP1', 'VP20', 'VP21', 'VP22', 'VP23', 'VP24', 'VP2', 'VP3', 'VP5', 'VP6', 'VP7', 'VP8', 'VP9']


In [36]:
# Merging the same events to a single sample number
def merge_events(events):
    events = mne.merge_events(events, [11,12,20,21,22,23,31,32], 11, replace_events=True) # stimulus/t1
    events = mne.merge_events(events, [64,65,66,67,68,128,129,130,131,132,133], 64, replace_events=True) # stimulus/t2
    events = mne.merge_events(events, [1,2,3], 1, replace_events=True) # response/t1
    events = mne.merge_events(events, [4,5], 4, replace_events=True) # response/t2
    events = mne.merge_events(events, [6,7], 6, replace_events=True) # block

    return events

In [37]:
# Decide if trial has a short or long SOA based on time difference of stimulus onsets
def find_condition(events):
    new_events = []
    existing_event_onsets_set1 = np.concatenate([events[events[:, 2] == 11, 0]]) # stimulus/t1
    existing_event_onsets_set2 = np.concatenate([events[events[:, 2] == 64, 0]]) # stimulus/t2
    
    for idx, x in enumerate(existing_event_onsets_set1):     
        # The time difference between the two stimuli is computed by finding for every stim1 the closest positive difference with all stim2
        diff = existing_event_onsets_set2 - x
        valid_indices = np.where(diff > 0)
        closest_index = np.argmin(diff[valid_indices])

        # Prevents error if there are more stimulus 1 events than stimulus 2 events
        if idx == len(existing_event_onsets_set2): break
        
        if diff[valid_indices][closest_index] > 600 and diff[valid_indices][closest_index] < 615: # to allow for some variation in the samples
            new_events.append([x, 0, 256])  # long delay
        elif diff[valid_indices][closest_index] > 140 and diff[valid_indices][closest_index] < 151:
            new_events.append([x, 0, 512])  # short delay
        else:
            new_events.append([x, 0, 1024])  # exclude
    return new_events

In [38]:
def get_events(raw, tmin, tmax):
    stimulus_id = {'stimulus/t1': 11, 'stimulus/t2': 64} # Stimulus trigger
    condition_id = {"condition/short": 512, "condition/long": 256} # Condition trigger
    response_id = {'response/t2': 4, 'response/t1': 1} # Response triggers
    event_id = condition_id | stimulus_id | response_id # All retained events

    events = mne.find_events(raw, shortest_event=1)
    events = merge_events(events)

    # Identify new events across trigger sets and consider only the two nearest events
    new_events = find_condition(events)
    new_events = np.array(new_events)
    events_temp = np.concatenate((events, new_events))

    # Get metadata and events
    metadata, events, event_id = mne.epochs.make_metadata(
        events=events_temp, event_id=event_id, tmin=tmin, tmax=tmax,
        sfreq=raw.info["sfreq"], row_events=list(stimulus_id.keys()), keep_first=["condition"])
    
    # Initialize trial index list and trial counter
    trial_indices = []
    trial_index = 0

    s1_indices = []
    s2_indices = []

    # Iterate through events and assign trial indices
    i = 0
    while i < len(events):
        if i < len(events) - 1 and ((events[i][2] == 11 and events[i + 1][2] == 64) or (events[i][2] == 64 and events[i + 1][2] == 11)):
            # Valid pair (11 -> 64 or 64 -> 11)
            trial_indices.append(trial_index)  # First event of the pair
            trial_indices.append(trial_index)  # Second event of the pair
            trial_index += 1  # Increment trial index for the next pair
            i += 2  # Move past the pair
        else:
            # Event doesn't form a valid pair, start a new trial for the next event
            trial_indices.append(trial_index)  # This event gets its own trial
            trial_index += 1  # Increment trial index for the next event
            i += 1  # Move to the next event
    
    metadata['trial_index'] = trial_indices

    for idx, e in enumerate(events):
        if e[2] == 11:
            s1_indices.append(idx)
        elif e[2] == 64:
            s2_indices.append(idx)

    return events, metadata, s1_indices, s2_indices

In [40]:
def create_epochs_t1(tmin, tmax, raw):
    events, metadata, s1_indices, _ = get_events(raw, tmin, tmax)
    metadata = metadata.iloc[s1_indices]
    # events = events[s1_indices]
    events = [events[i] for i in s1_indices]
    
    keep_cols = ['response/t1', 'first_condition', 'trial_index']
    metadata = metadata[keep_cols]
    metadata.reset_index(drop=True, inplace=True) # This allows to preserve the trial number after the rejections performed by the mne.Epochs function
    metadata.columns = ['rt', 'condition', 'trial_index'] # More convenient names

    event_id = {'stimulus/t1': 11} # S1 will be the onset of the epochs

    # Create epochs
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                baseline=(None, 0), preload=True, 
                verbose=True, detrend=1, on_missing = 'warn', event_repeated='drop',
                metadata=metadata, reject_by_annotation=True, reject=None)
    
    return epochs, events

In [41]:
def create_epochs_t2(tmin, tmax, raw):
    events, metadata, s1_indices, s2_indices = get_events(raw, tmin, tmax)
    metadata_2 = metadata.iloc[s1_indices]
    metadata = metadata.iloc[s2_indices]
    events = [events[i] for i in s2_indices]
    
    # Move up RTs of S2 based on short/long SOA because they are originally based on S1
    metadata_2['response/t2'] = np.where(metadata_2['first_condition']=='short', metadata_2['response/t2']-0.3, metadata_2['response/t2']-1.2)
    metadata['response/t2'] = metadata['response/t2'] - 0.024
    metadata.reset_index(drop=True, inplace=True)
    metadata_2.reset_index(drop=True, inplace=True)

    # Create a mapping from trial_index to first_condition from metadata_2 and update first_condition column in metadata only where trial_index matches
    update_map = metadata_2.set_index('trial_index')['first_condition']
    metadata['first_condition'] = metadata['trial_index'].map(update_map).fillna(metadata['first_condition'])

    update_map = metadata_2.set_index('trial_index')['response/t2']
    metadata['response/t2'] = metadata['trial_index'].map(update_map).fillna(metadata['response/t2'])
    
    keep_cols = ['response/t2', 'first_condition', 'trial_index']
    metadata = metadata[keep_cols]
    metadata.reset_index(drop=True, inplace=True) # This allows to preserve the trial number after the rejections performed by the mne.Epochs function
    metadata.columns = ['rt', 'condition', 'trial_index'] # More convenient names

    event_id = {'stimulus/t2': 64} # S2 will be the onset of the epochs

    # Create epochs
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                baseline=(None, 0), preload=True, 
                verbose=True, detrend=1, on_missing = 'warn', event_repeated='drop',
                metadata=metadata, reject_by_annotation=True, reject=None)
    
    return epochs, events

In [42]:
def create_epochs_trial(tmin, tmax, raw):
    events, metadata, s1_indices, _ = get_events(raw, tmin, tmax)
    metadata = metadata.iloc[s1_indices]
    # events = events[s1_indices]
    events = [events[i] for i in s1_indices]
    
    keep_cols = ['response/t2', 'first_condition', 'trial_index']
    metadata = metadata[keep_cols]
    metadata.reset_index(drop=True, inplace=True) # This allows to preserve the trial number after the rejections performed by the mne.Epochs function
    metadata.columns = ['rt', 'condition', 'trial_index'] # More convenient names

    event_id = {'stimulus/t1': 11} # S1 will be the onset of the epochs

    # Create epochs
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                baseline=(None, 0), preload=True, 
                verbose=True, detrend=1, on_missing = 'warn', event_repeated='drop',
                metadata=metadata, reject_by_annotation=True, reject=None)
    
    return epochs, events

In [43]:
def autoreject(epochs, raw):
    picks = mne.pick_types(raw.info, eeg=False, stim=False, eog=False, include=['C3', 'C4', 'POz', 'FCz'], exclude=[])
    ar = aj.AutoReject()
    
    # Fit on the first 20 epochs to save time
    ar.fit(epochs[0:20])
    epochs_ar, reject_log = ar.transform(epochs, return_log=True)

    return epochs_ar

In [ ]:
tmin = -0.25
tmax = 2

for subj in subj_names:
    # Skip those subjects because the data is not valid
    if subj == 'VP6' or subj == 'VP7':
        continue

    raw = mne.io.read_raw_fif(path+"Post ICA/post_ica_%s_raw.fif"%subj, preload=True)
    # epoch = mne.read_epochs(epoch_data_path+subj+'.fif')

    # ================= STIMULUS 1 =================
    epochs, events = create_epochs_t1(tmin, tmax, raw)
    epochs = autoreject(epochs, raw)
    epochs.save(path+'Stimlocked_fixed_index/%s_epo_t1.fif'%subj, overwrite=True)

    # # ================= STIMULUS 2 =================
    epochs, events = create_epochs_t2(tmin, tmax, raw)
    epochs = autoreject(epochs, raw)
    epochs.save(path+'Stimlocked_fixed_index/%s_epo_t2.fif'%subj, overwrite=True)

    # ================= WHOLE TRIALS =================
    # epochs, events = create_epochs_trial(tmin, tmax, raw)
    # epochs = autoreject(epochs, raw)
    # epochs.save(path+'Stimlocked_fixed_index/%s_epo_trial.fif'%subj, overwrite=True)

Opening raw data file C:/Users/annad/Documents/Thesis/Data/Post ICA/post_ica_VP10_raw.fif...
    Range : 0 ... 2022399 =      0.000 ...  3949.998 secs
Ready.
Reading 0 ... 2022399  =      0.000 ...  3949.998 secs...
Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
6582 events found on stim channel Status
Event IDs: [  1   2   3   4   5   6  11  12  16  21  22  31  32  64  66  67 128 129
 130]
Adding metadata with 3 columns
1309 matching events found
Setting baseline interval to [-0.25, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 1309 events and 1153 original time points ...
12 bad epochs dropped
Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/64 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/64 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/20 [00:00<?,       ?it/s]

  0%|          | n_interp : 0/3 [00:00<?,       ?it/s]

...





Estimated consensus=0.90 and n_interpolate=32


  0%|          | Repairing epochs : 0/1268 [00:00<?,       ?it/s]

Dropped 42 epochs: 26, 27, 31, 58, 63, 66, 74, 91, 98, 116, 123, 125, 126, 156, 161, 171, 184, 189, 208, 249, 296, 328, 350, 374, 385, 390, 416, 434, 447, 638, 662, 691, 713, 749, 812, 833, 869, 874, 877, 895, 918, 940
Overwriting existing file.


C:\Users\annad\AppData\Local\Temp\ipykernel_172744\2105183387.py:20: RuntimeWarning: This filename (C:/Users/annad/Documents/Thesis/Data/Stimlocked_fixed_index/VP9_epo_t2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(path+'Stimlocked_fixed_index/%s_epo_t2.fif'%subj, overwrite=True)


Overwriting existing file.
